In [24]:
#MERCARI PRICE SUGGESTION

In [2]:
#importing required libraries
import dask.dataframe as dd
from dask_ml.model_selection import GridSearchCV
from dask_ml.model_selection import train_test_split
import numpy as np
import dask.array as da


In [3]:
#Start Dask Client for Dashboard
from dask.distributed import Client, progress
client = Client(threads_per_worker=1,
                n_workers=4,
                memory_limit='2GB')
client

Client Scheduler: tcp://127.0.0.1:34381 Dashboard: /user/dask-dask-examples-lv1c0e6c/proxy/8787/status,Cluster Workers: 4 Cores: 4 Memory: 8.00 GB


In [4]:
#Reading Train data
df= dd.read_csv('https://raw.githubusercontent.com/bethtseng/Price_Suggestion/master/sample_data/train.tsv',delimiter='\t',blocksize=None)

In [5]:
df.head()

,train_id,name,item_condition_id,category_name,brand_name,price,shipping,item_description
0,0,MLB Cincinnati Reds T Shirt Size XL,3,Men/Tops/T-shirts,NaN,10.0,1,No description yet
1,1,Razer BlackWidow Chroma Keyboard,3,Electronics/Computers & Tablets/Components & P...,Razer,52.0,0,This keyboard is in great condition and works ...
2,2,AVA-VIV Blouse,1,Women/Tops & Blouses/Blouse,Target,10.0,1,Adorable top with a hint of lace and a key hol...
3,3,Leather Horse Statues,1,Home/Home Décor/Home Décor Accents,NaN,35.0,1,New with tags. Leather horses. Retail for [rm]...
4,4,24K GOLD plated rose,1,Women/Jewelry/Necklaces,NaN,44.0,0,Complete with certificate of authenticity


In [6]:
df['item_description'].fillna(value="Missing")

Dask Series Structure:
npartitions=1
    object
       ...
Name: item_description, dtype: object
Dask Name: fillna, 5 tasks

In [7]:
X = (df['name']+ ' '+df['item_description'])
y = np.log1p(df['price'])


In [8]:
import re
def tokenizer(text):
    if text:
        result = re.findall('[a-z]{2,}', text.lower())
    else:
        result = []
    return result


In [9]:
X_train, X_test, y_train, y_test= train_test_split(X, y, test_size=0.3, random_state=101)

In [10]:
#training tfidf vectorizer on the combination of the item name and description.
import sklearn.feature_extraction.text as tdd
vect = tdd.TfidfVectorizer(tokenizer=tokenizer,stop_words='english')


In [11]:
X_Train_vect=vect.fit_transform(X_train.compute()).todense()


In [12]:
#using the SGDRegressor with ordinary least squares.
#using mean squared error as the eval metric, since negative values crash mean squared log error.
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_log_error
from sklearn.linear_model import SGDRegressor

seed=101

In [13]:
model = SGDRegressor(loss='squared_loss', penalty='l2', random_state=seed, max_iter=5)
params = {'penalty':['none','l2','l1'],
          'alpha':[1e-4, 2e-4, 5e-4, 1e-3, 2e-3, 5e-3, 1e-2, 2e-2, 5e-2, 0.1]}

In [14]:
gs = GridSearchCV(estimator=model,
                  param_grid=params,
                  scoring='neg_mean_squared_error',
                  n_jobs=1,
                  cv=5,
                  verbose=3)


In [15]:
gs.fit(X_Train_vect, y_train.compute())


Fitting 5 folds for each of 30 candidates, totalling 150 fits
[CV] alpha=0.0001, penalty=none ......................................
[CV]  alpha=0.0001, penalty=none, score=-0.700505196286401, total=   0.1s
[CV] alpha=0.0001, penalty=none ......................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
/srv/conda/envs/notebook/lib/python3.7/site-packages/sklearn/linear_model/stochastic_gradient.py:183: FutureWarning: max_iter and tol parameters have been added in SGDRegressor in 0.19. If max_iter is set but tol is left unset, the default value for tol in 0.19 and 0.20 will be None (which is equivalent to -infinity, so it has no effect) but will change in 0.21 to 1e-3. Specify tol to silence this warning.
  FutureWarning)
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.2s remaining:    0.0s
/srv/conda/envs/notebook/lib/python3.7/site-packages/sklearn/linear_model/stochastic_gradient.py:183: FutureWarning: max_iter and tol parameters have been added in SGDRegressor in 0.19. If max_iter is set but tol is left unset, the default value for tol in 0.19 and 0.20 will be None (which is equivalent to -infinity, so it has no effect) but will change in 0.21 to 1e-3. Specify tol to silence this warning.
  

[CV]  alpha=0.0001, penalty=none, score=-0.7042286956319315, total=   0.2s
[CV] alpha=0.0001, penalty=none ......................................
[CV]  alpha=0.0001, penalty=none, score=-0.6024526179684444, total=   0.2s
[CV] alpha=0.0001, penalty=none ......................................


/srv/conda/envs/notebook/lib/python3.7/site-packages/sklearn/linear_model/stochastic_gradient.py:183: FutureWarning: max_iter and tol parameters have been added in SGDRegressor in 0.19. If max_iter is set but tol is left unset, the default value for tol in 0.19 and 0.20 will be None (which is equivalent to -infinity, so it has no effect) but will change in 0.21 to 1e-3. Specify tol to silence this warning.
  FutureWarning)
/srv/conda/envs/notebook/lib/python3.7/site-packages/sklearn/linear_model/stochastic_gradient.py:183: FutureWarning: max_iter and tol parameters have been added in SGDRegressor in 0.19. If max_iter is set but tol is left unset, the default value for tol in 0.19 and 0.20 will be None (which is equivalent to -infinity, so it has no effect) but will change in 0.21 to 1e-3. Specify tol to silence this warning.
  FutureWarning)


[CV]  alpha=0.0001, penalty=none, score=-0.3772323429252322, total=   0.1s
[CV] alpha=0.0001, penalty=none ......................................
[CV]  alpha=0.0001, penalty=none, score=-0.6935193399246564, total=   0.2s
[CV] alpha=0.0001, penalty=l2 ........................................
[CV]  alpha=0.0001, penalty=l2, score=-0.7005080778824991, total=   0.1s
[CV] alpha=0.0001, penalty=l2 ........................................


...

[CV] . alpha=0.1, penalty=l1, score=-0.3805256444739548, total=   0.3s
[CV] alpha=0.1, penalty=l1 ...........................................


/srv/conda/envs/notebook/lib/python3.7/site-packages/sklearn/linear_model/stochastic_gradient.py:183: FutureWarning: max_iter and tol parameters have been added in SGDRegressor in 0.19. If max_iter is set but tol is left unset, the default value for tol in 0.19 and 0.20 will be None (which is equivalent to -infinity, so it has no effect) but will change in 0.21 to 1e-3. Specify tol to silence this warning.
  FutureWarning)


[CV] . alpha=0.1, penalty=l1, score=-0.7025914543343653, total=   0.3s


[Parallel(n_jobs=1)]: Done 150 out of 150 | elapsed:   41.8s finished
/srv/conda/envs/notebook/lib/python3.7/site-packages/sklearn/model_selection/_search.py:842: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)
/srv/conda/envs/notebook/lib/python3.7/site-packages/sklearn/linear_model/stochastic_gradient.py:183: FutureWarning: max_iter and tol parameters have been added in SGDRegressor in 0.19. If max_iter is set but tol is left unset, the default value for tol in 0.19 and 0.20 will be None (which is equivalent to -infinity, so it has no effect) but will change in 0.21 to 1e-3. Specify tol to silence this warning.
  FutureWarning)


GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=SGDRegressor(alpha=0.0001, average=False, early_stopping=False, epsilon=0.1,
       eta0=0.01, fit_intercept=True, l1_ratio=0.15,
       learning_rate='invscaling', loss='squared_loss', max_iter=5,
       n_iter=None, n_iter_no_change=5, penalty='l2', power_t=0.25,
       random_state=101, shuffle=True, tol=None, validation_fraction=0.1,
       verbose=0, warm_start=False),
       fit_params=None, iid='warn', n_jobs=1,
       param_grid={'penalty': ['none', 'l2', 'l1'], 'alpha': [0.0001, 0.0002, 0.0005, 0.001, 0.002, 0.005, 0.01, 0.02, 0.05, 0.1]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='neg_mean_squared_error', verbose=3)

In [16]:
model = gs.best_estimator_

In [17]:
print(gs.best_params_)
print(gs.best_score_)


{'alpha': 0.0001, 'penalty': 'none'}
-0.6158297007716338


In [18]:
# package everything up in a pipeline and run it over the test set to check the performance.
from sklearn.pipeline import Pipeline
pipe = Pipeline([('vect',vect),('model',gs)])
y_pred =pipe.predict(X_test.compute())


In [19]:
# Replace negative values with zero for the time being.
print(np.sqrt(mean_squared_log_error(np.exp(y_test)-1, np.exp(y_pred)-1)))

0.7205414988698174


In [20]:
#reading test data
df_test= dd.read_csv('https://raw.githubusercontent.com/bethtseng/Price_Suggestion/master/sample_data/test.tsv',delimiter='\t',blocksize= None,engine='python',quoting=3, error_bad_lines=False)


In [21]:
df_test['item_description'].fillna('Missing')


Dask Series Structure:
npartitions=1
    object
       ...
Name: item_description, dtype: object
Dask Name: fillna, 5 tasks

In [22]:
import dask.array as da
ffss=dd.from_dask_array(da.from_array(np.exp(pipe.predict((df_test['name'] + ' ' + df_test['item_description']).compute()))-1))
type(ffss)
df_test_new=df_test.reset_index().set_index('index')
final=df_test_new.assign(price=ffss)

In [23]:
final.head(10)

,test_id,name,item_condition_id,category_name,brand_name,shipping,item_description,price
index,,,,,,,,
0,0,"""Breast cancer """"I fight like a girl"""" ring""",1,Women/Jewelry/Rings,NaN,1,Size 7,19.205672
1,1,"""25 pcs NEW 7.5""""x12"""" Kraft Bubble Mailers""",1,Other/Office supplies/Shipping Supplies,NaN,1,"""25 pcs NEW 7.5""""x12"""" Kraft Bubble Mailers Li...",18.570315
2,2,Coach bag,1,Vintage & Collectibles/Bags and Purses/Handbag,Coach,1,Brand new coach bag. Bought for [rm] at a Coac...,19.121575
3,3,Floral Kimono,2,Women/Sweaters/Cardigan,NaN,0,-floral kimono -never worn -lightweight and pe...,18.550305
4,4,Life after Death,3,Other/Books/Religion & Spirituality,NaN,1,Rediscovering life after the loss of a loved o...,18.654247
5,5,iPhone 6 Plus or 6s Plus Vodka pink case,1,"Electronics/Cell Phones & Accessories/Cases, C...",NaN,1,One Absolut Vodka in Pink for iPhone 6 Plus an...,19.208415
6,6,Vintage Cameo Pendant & Brooch Pin,3,Women/Jewelry/Necklaces,Vintage,1,"""Two vintage Cameo pieces. 1. Silver metal Loc...",18.554578
7,7,Rose Gold Stainless Steel Quartz Watch,1,Women/Women's Accessories/Watches,NaN,1,"""Brand new Price firm No trades Box included w...",18.884453
8,8,Daisy Marc Jacobs 3.4oz,3,Beauty/Fragrance/Women,MARC JACOBS,0,Brand new No box 100% authentic Firm price NO ...,20.191782
